In [1]:
import re
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.DataStructs.cDataStructs import ConvertToNumpyArray
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from rdkit.Chem.Scaffolds.MurckoScaffold import MurckoScaffoldSmiles, MurckoScaffoldSmilesFromSmiles
from rdkit.DataStructs.cDataStructs import ExplicitBitVect, TanimotoSimilarity
import chromadb
from chromadb import Client, Settings, EmbeddingFunction
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOpenAI
from langchain.schema import HumanMessage 
import statistics
import random
from langchain_community.chat_models import ChatOllama

# Env

In [2]:
import os
os.environ["OPENAI_API_KEY"] = ""

# Config

In [20]:
radius=3
nBits=2048
llm_model = 'gpt-3.5-turbo'
num_llm_calls = 3
context_window = 20
template = "{context}. As a language model you have an ability to understand this data based on the data provided, we do not need to build a regression model for a Language model Estimate. You can find relationships and make estimates. Make an estimate for Permeabilty based on SMILES and list of amino acids and on previous data for this row of data {query}. Report the predicted permeability value in the following format: PERMEABILITY(%value)."

# Utils


In [4]:
def get_morgan_fingerprint(smiles, radius=radius, nBits=nBits):
    mol = Chem.MolFromSmiles(smiles)
    morgan_fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits)
    arr = np.zeros((1,), dtype=np.int8)
    ConvertToNumpyArray(morgan_fp, arr)
    return arr.tolist()


def find_smiles(text):
    pattern = r'SMILES{([^}]*)}'
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    else:
        return None
    
def chemical_embedding(text):
    return get_morgan_fingerprint(find_smiles(text), radius, nBits)

In [5]:

class ChemicalEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input):
        smiles = self.find_smiles(input)
        if smiles:
            return self.get_morgan_fingerprint(smiles)
        return None
    
    def embed_query(self, query):
        # This method should handle the embedding of queries
        smiles = self.find_smiles(query)
        if smiles:
            return self.get_morgan_fingerprint(smiles)
        return None

    def find_smiles(self, text):
        pattern = r'SMILES{([^}]*)}'
        match = re.search(pattern, text)
        if match:
            return match.group(1)
        else:
            return None

    def get_morgan_fingerprint(self, smiles, radius=radius, nBits=nBits):
        mol = Chem.MolFromSmiles(smiles)
        morgan_fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits)
        arr = np.zeros((1,), dtype=np.int8)
        ConvertToNumpyArray(morgan_fp, arr)
        return arr.tolist()

In [6]:
def convert_aldehydes_to_acids(smiles_list):
    rxn_smarts = '[CX3H1:1](=O)[H].[OH2:2]>>[CX3:1](=O)[O:2]'
    rxn = AllChem.ReactionFromSmarts(rxn_smarts)
    
    acid_smiles_list = []
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        mol = Chem.AddHs(mol)
        ps = rxn.RunReactants((mol, Chem.MolFromSmiles('O')))
        if ps:
            product_mol = ps[0][0]  
            Chem.SanitizeMol(product_mol)  
            acid_smiles = Chem.MolToSmiles(Chem.RemoveHs(product_mol), isomericSmiles=True, canonical=True)
            acid_smiles_list.append(acid_smiles)
        else:
            print('Issue with converting C=O into -COOH')
            acid_smiles_list.append(smiles)
    
    return acid_smiles_list

def tokenize_peptides(smiles):
    mol = Chem.MolFromSmiles(smiles)
    pat = Chem.MolFromSmarts('NC=O')
    
    # Finding the largest ring
    def find_largest_ring(mol):
        sssr = Chem.GetSymmSSSR(mol)
        largest_ring = max(sssr, key=len)
        return set(largest_ring)

    largest_ring = find_largest_ring(mol)
    matches = mol.GetSubstructMatches(pat)
    
    emol = Chem.EditableMol(mol)

    bonds_to_break = []

    for match in matches:
        N_idx, C_idx, O_idx = match
        if N_idx in largest_ring and C_idx in largest_ring:
            bonds_to_break.append((N_idx, C_idx))

    # Break bonds 
    for N_idx, C_idx in sorted(bonds_to_break, reverse=True):  # Sort and reverse to avoid indexing issues
        emol.RemoveBond(N_idx, C_idx)

    fragmented_mol = emol.GetMol()
    frags = Chem.GetMolFrags(fragmented_mol, asMols=True, sanitizeFrags=True)
    fragment_smiles = [Chem.MolToSmiles(frag) for frag in frags]

    return convert_aldehydes_to_acids(fragment_smiles)

# Read csv

In [7]:
train_set = pd.read_csv('../data/train_set.csv')
test_set = pd.read_csv('../data/test_set.csv')
val_set = pd.read_csv('../data/val_set.csv')

# Chemical VDB generation

In [8]:

class ChemicalEmbeddingGenerator:

    @staticmethod
    def generate_embedding(chunk):
        def extract_smiles(text):
            pattern = r'SMILES{([^}]*)}'
            match = re.search(pattern, text)
            if match:
                return match.group(1)
            return None
        smiles_string = extract_smiles(chunk)
        try:
            mol = Chem.MolFromSmiles(smiles_string)
            if mol is not None:
                morgan_fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=nBits)
                arr = np.zeros((1,), dtype=np.int8)
                ConvertToNumpyArray(morgan_fp, arr)
                #print("Embedding generated successfully.")
                return arr.tolist()
            else:
                print("Invalid SMILES string.")
                return None
        except Exception as e:
            print(f"An error occurred while generating embedding: {e}")
            return None
        


class DataFrameManager:
    def __init__(self):
        self.embedding_generator = ChemicalEmbeddingGenerator()
        self.chroma_client = chromadb.chromadb.Client()
        self.collection_name = "chemical_data_collection"
        self.collection = self.chroma_client.get_or_create_collection(name=self.collection_name,
                                                                      metadata={"hnsw:space": "cosine"})
        
    def add_texts_to_collection(self, dataframe, text_column, columns_to_keep):
        embeddings_list = []
        documents_list = []
        metadatas_list = []
        ids_list = []
        
        for index, row in dataframe.iterrows():
            embedding = self.embedding_generator.generate_embedding(row[text_column])
            if embedding is not None:
                unique_id = f"row_{index}"
                embeddings_list.append(embedding)
                documents_list.append(row[text_column])

                metadata = {column: row[column] for column in columns_to_keep if row[column] is not None}
                
                metadatas_list.append(metadata)
                ids_list.append(unique_id)
        
        self.collection.add(
            embeddings=embeddings_list,
            documents=documents_list,
            metadatas=metadatas_list,
            ids=ids_list
        )
        print("Data added to collection successfully.")
        return self.collection


In [9]:
def get_context(query_dict_output):
    context=''
    for instance in query_dict_output['documents'][0]:
        context+=instance + '. '
    return context

def get_distances(query_dict_output):
    return query_dict_output['distances'][0]

In [10]:
def extract_value(input_string):
    match = re.search(r'PERMEABILITY\(([^)]+)\)', input_string)

    if match:
        # Extracting the value
        permeability_value = match.group(1)
    else:
        permeability_value = None

    return float(permeability_value)

In [11]:
def get_values_from_context(input_string):

    match = re.search(r'permeability value is ([\-\d\.]+)', input_string)
    permeability_value = match.group(1)

    return float(permeability_value)

def context_values(context_dict):
    permeability = []
    for sample in context_dict['documents'][0]:
        permeability.append(get_values_from_context(sample))
    return permeability

def context_stats(context_permeability_list):
    ''' The first value is mean, the second one is the closest by Tanimoto structure'''
    return statistics.fmean(context_permeability_list), context_permeability_list[0] 

In [12]:
def find_closest(target, numbers):
    return min(numbers, key=lambda x: abs(x - target))

In [13]:
def calculate_errors(exact_value, approximate_value):
    absolute_error = abs(exact_value - approximate_value)
    if exact_value != 0:  
        relative_error = (absolute_error / abs(exact_value)) 
    else:
        relative_error = None  
    return round(relative_error, 3)

In [14]:
def calculate_accuracy(exact_value, approximate_value):
    if exact_value != 0:  
        accuracy = (1 - abs(exact_value - approximate_value) / abs(exact_value))
    else:
        accuracy = None
    return round(accuracy, 3)

In [16]:
def extract_initial_data(row):
    data = {
        'text': [row['Formatted_String']],
        'ground_truth': [row['PAMPA']],
        'tanimoto': [row['Mean_Tanimoto_Similarity']],
        'structure': [row['SMILES']],
        'aminoacids': [row['tokens']],
    }
    return data

def vdb_initialization(text):
    sample_from_chroma = manager.collection.query(query_embeddings=[chemical_embedding(text)], n_results=context_window)
    return sample_from_chroma

def context_metrics(vdb_instance):
    retrieved_values = context_values(vdb_instance)
    context_mean_perm, context_top_1_perm = context_stats(retrieved_values)
    context_mean_distance, context_closest_distance = context_stats(get_distances(vdb_instance))
    context_statistics = {'Context Mean Permeability': [context_mean_perm],
                    'Context Top 1 Permeability': [context_top_1_perm],
                    'Context Mean Distance': [context_mean_distance],
                    'Context Closest Distance': [context_closest_distance]
                    }
    return context_statistics

def query_openai_model(vdb_instance, text):
    prompt = template.format(context=get_context(vdb_instance), query=text)
    openai_llm = ChatOpenAI(model_name=llm_model)
    predicted_values = []
    responses = []
    
    for i in range(num_llm_calls):
        response = openai_llm([HumanMessage(content=prompt)])
        token_stats = response.response_metadata['token_usage']
        predicted_value = extract_value(response.content)
        predicted_values.append(predicted_value)
        responses.append(response.content)

    n_input_tokens = token_stats['prompt_tokens']
    n_of_examples = context_window
    data = {'Predicted Values': [predicted_values],
            'Number of samples': [n_of_examples],
            'Prompt tokens': [n_input_tokens]     
            }
    return data

def query_randomizer(vdb_instance):
    global llm_model
    llm_model = 'random'
    predicted_values = []
    for i in range(num_llm_calls):
        predicted_values.append(random.choice(context_values(vdb_instance)))
    n_of_examples = context_window
    data = {'Predicted Values': [predicted_values],
        'Number of samples': [n_of_examples]  
        }
    return data

def llm_output_metrics(predicted_values, ground_truth):
    mean_llm_response = round(statistics.fmean(predicted_values),2)
    closest_guess = find_closest(ground_truth, predicted_values)

    std = round(np.std(predicted_values), 4) 
    error_mean = calculate_errors(ground_truth, mean_llm_response)
    error_closest = calculate_errors(ground_truth, closest_guess)
    acc_mean = calculate_accuracy(ground_truth, mean_llm_response)
    acc_closest = calculate_accuracy(ground_truth, closest_guess)

    llm_metrics = {
        'Mean LLM Response': [mean_llm_response],
        'Closest Guess': [closest_guess],
        'Standard Deviation': [std],
        'Error Mean': [error_mean],
        'Error Closest': [error_closest],
        'Accuracy Mean': [acc_mean],
        'Accuracy Closest': [acc_closest]
        }
    return llm_metrics

def setup_parameters():
    return {'Radius': [radius],
        'nBits': [nBits],
        'Model': [llm_model]}


def run_evaluation(row, type_of_model='gpt'):
    initial_data = extract_initial_data(row)
    vdb = vdb_initialization(initial_data['text'][0])
    if type_of_model == 'gpt':
        llm_output = query_openai_model(vdb, initial_data['text'][0])
    elif type_of_model == 'random':
        print('random model is used')
        llm_output = query_randomizer(vdb)
    context_metr = context_metrics(vdb)
    llm_response_metrics = llm_output_metrics(llm_output['Predicted Values'][0], initial_data['ground_truth'][0])
    setup_params = setup_parameters()
    final_dict = initial_data | context_metr | llm_output | llm_response_metrics | setup_params
    return pd.DataFrame(final_dict)

In [17]:
def main(df, type_of_model, type_of_test):
    global manager
    manager = DataFrameManager()
    manager.add_texts_to_collection(train_set, 'Formatted_String', ['Formatted_String'])
    df_list = []
    for index, row in df.iterrows():
        df_list.append(run_evaluation(row, type_of_model=type_of_model))
    manager.chroma_client.delete_collection(name='chemical_data_collection')
    del manager
    return pd.concat(df_list).to_csv(f'../results/{llm_model}_{context_window}_{nBits}_{radius}_{type_of_test}.csv')
 

# Refining functions

In [21]:
main(validation, type_of_model='random', type_of_test='val')

Data added to collection successfully.
random model is used
random model is used
